In [1]:
import numpy as np # For calculations

from sklearn.datasets import load_iris  # Imported iris dataset

iris = load_iris()
data = iris.data # Features
target = iris.target # Corresponding targets

In [2]:
# for extracting number of 0,1,2 classes from given set of datapoints
def num_class(index):
    z = 0
    o = 0
    t = 0
    for i in index:
        x = target[i]
        if x == 0:
            z += 1
        elif x == 1:
            o += 1
        elif x == 2:
            t += 1
    return z,o,t,z+o+t

In [3]:
# For calculating Entropy on basis of number of 0,1,2 classes
def Entropy(z,o,t,tt):
    en = 0
    for i in (z,o,t):
        if i != 0:
            p = i/tt
            en -= p*np.log2(p)
    return en

In [4]:
# Created Node class for nodes in Decision Tree
class Node:
    def __init__(self,index,feature,en):
        # I am keeping track of only index of datapoints for each nodes to avoid copying data again and again
        self.index = index 
        self.z,self.o,self.t,self.tt = num_class(index) # Number of each class 0,1,2 the node have
        self.feature = feature # Features on which present node can be further splitted
        self.c1 = None
        self.c2 = None
        self.entropy = en
        self.gain_ratio= None
        
    # For Printing Content of the node
    def display(self):
        if self.z != 0:
            print("Count of 0 =",self.z)
        if self.o != 0:
            print("Count of 1 =",self.o)
        if self.t != 0:
            print("Count of 2 =",self.t)
        print("Current Entropy is =",self.entropy)
        

In [5]:
class Decision_Tree:
    def __init__(self,data,target):
        self.features = ['x1','x2','x3','x4']
        self.data = data
        self.target = target
        self.root = Node([x for x in range(data.shape[0])],[x for x in range(data.shape[1])],Entropy(*num_class(range(data.shape[0]))))
        # Above one is Creation of Root Node
    
    # Following function Decides, at which point datapoints should be splitted on particular feature
    # to get maximum Entropy    
    def feature_modifier(self,index,f):
        sep = []
        for i in index:
            x = data[i][f] + 0.0001
            if x not in sep:
                sep.append(x)
        
        sep.remove(max(sep))
           
        gen = 9
        gena = 0
        genb = 0
        ga = []
        gb = []
        
        for s in sep:
            a = []
            b = []
            for i in index:
                x = data[i][f]
                
                if x < s:
                    a.append(i)
                else:
                    b.append(i)
            la = len(a)
            lb = len(b)
         
            lab = la+lb
            ena = Entropy(*num_class(a))
            enb = Entropy(*num_class(b))
            en = ena*la/lab + enb*lb/lab
            if gen > en:
                gen = en
                ga = a
                gb = b
                gena = ena
                genb = enb
             
        return ga,gb,gen,gena,genb
    
    # Decide Best Feature to Split upon
    def splitter(self,node):
        F = node.feature
        if len(F) == 0:
            return 
        gena = 0
        genb = 0
        gen = 9
        ga = []
        gb = []
        fr = None
        for f in F:
            a,b,en,ena,enb = self.feature_modifier(node.index,f)
            if gen > en:
                gen = en
                gena = ena
                genb = enb
                ga = a
                gb = b
                fr = f
        
        F.remove(fr)
        Fc = F.copy()
        node.c1 = Node(ga,F,gena)
        node.c2 = Node(gb,Fc,genb)    
        node.gain_ratio = node.entropy - gen
        print('Splitting on feature',self.features[fr],'with gain ratio',node.gain_ratio)
    
    def start(self,node,L = 0):
        if node is None: # Exit if Node is None
            return 
        print("Level",L)
        node.display()
        
        if node.entropy == 0 or len(node.feature) == 0: # Exit if Entropy is 0 or all Featured used up
            print('Reached leaf Node')
            print()
            return 
        self.splitter(node) # Split the Node
        print()
        self.start(node.c1,L+1) # Recursion call on first child
        self.start(node.c2,L+1) # Recursion call of second child
        

In [6]:
tree = Decision_Tree(data,target) # Decision Tree Initialization 

In [7]:
tree.start(tree.root) #Recursively Start Splitting each Node 

Level 0
Count of 0 = 50
Count of 1 = 50
Count of 2 = 50
Current Entropy is = 1.584962500721156
Splitting on feature x3 with gain ratio 0.9182958340544894

Level 1
Count of 0 = 50
Current Entropy is = 0.0
Reached leaf Node

Level 1
Count of 1 = 50
Count of 2 = 50
Current Entropy is = 1.0
Splitting on feature x4 with gain ratio 0.6901603707546748

Level 2
Count of 1 = 49
Count of 2 = 5
Current Entropy is = 0.44506485705083865
Splitting on feature x1 with gain ratio 0.06619445463964702

Level 3
Count of 1 = 49
Count of 2 = 4
Current Entropy is = 0.3860189005698934
Splitting on feature x2 with gain ratio 0.06152601257782775

Level 4
Count of 1 = 27
Count of 2 = 4
Current Entropy is = 0.5547781633412736
Reached leaf Node

Level 4
Count of 1 = 22
Current Entropy is = 0.0
Reached leaf Node

Level 3
Count of 2 = 1
Current Entropy is = 0.0
Reached leaf Node

Level 2
Count of 1 = 1
Count of 2 = 45
Current Entropy is = 0.15109697051711368
Splitting on feature x1 with gain ratio 0.0610598085589334

## Second Approach
### Splitting on basis of Mean of Particular Feature

In [8]:
class Decision_Tree:
    def __init__(self,data,target):
        self.features = ['x1','x2','x3','x4']
        self.data = data
        self.target = target
        self.root = Node([x for x in range(data.shape[0])],[x for x in range(data.shape[1])],Entropy(*num_class(range(data.shape[0]))))
        # Above one is Creation of Root Node
    
    # Following function Just Decides to Split on mean of particular feature
    def feature_modifier(self,index,f):
        s = 0
        for i in index:
            s += data[i][f]
        s = s/len(index)
        
        a = []
        b = []
        for i in index:
            x = data[i][f]
            
            if x < s:
                a.append(i)
            else:
                b.append(i)
        la = len(a)
        lb = len(b)
        
        lab = la+lb
        ena = Entropy(*num_class(a))
        enb = Entropy(*num_class(b))
        en = ena*la/lab + enb*lb/lab
       
        
        return a,b,en,ena,enb
    
    # Decide Best Feature to Split upon
    def splitter(self,node):
        F = node.feature
        
        if len(F) == 0:
            return 
        gena = 0
        genb = 0
        gen = 9
        ga = []
        gb = []
        fr = None
        for f in F:
            a,b,en,ena,enb = self.feature_modifier(node.index,f)
            if gen > en:
                gen = en
                gena = ena
                genb = enb
                ga = a
                gb = b
                fr = f
                
       
        F.remove(fr)
        Fc = F.copy()
        node.c1 = Node(ga,F,gena)
        node.c2 = Node(gb,Fc,genb)    
        node.gain_ratio = node.entropy - gen
        print('Splitting on feature',self.features[fr],'with gain ratio',node.gain_ratio)
    
    def start(self,node,L = 0):
        
        if node is None: # Exit if Node is None
            return 
        print("Level",L)
        node.display()
        
        if node.entropy == 0 or len(node.feature) == 0:
            print('Reached leaf Node')
            print()
            return 
        self.splitter(node)# Split the Node
        print()
        self.start(node.c1,L+1)# Recursion call on first child
        self.start(node.c2,L+1)# Recursion call on second child
        

In [9]:
tree = Decision_Tree(data,target) # Decision Tree Initialization 

In [10]:
tree.start(tree.root)#Recursively Start Splitting each Node 

Level 0
Count of 0 = 50
Count of 1 = 50
Count of 2 = 50
Current Entropy is = 1.584962500721156
Splitting on feature x3 with gain ratio 0.7632957516786808

Level 1
Count of 0 = 50
Count of 1 = 7
Current Entropy is = 0.5373760853377336
Splitting on feature x4 with gain ratio 0.25984642687078696

Level 2
Count of 0 = 41
Current Entropy is = 0.0
Reached leaf Node

Level 2
Count of 0 = 9
Count of 1 = 7
Current Entropy is = 0.9886994082884974
Splitting on feature x2 with gain ratio 0.9886994082884974

Level 3
Count of 1 = 7
Current Entropy is = 0.0
Reached leaf Node

Level 3
Count of 0 = 9
Current Entropy is = 0.0
Reached leaf Node

Level 1
Count of 1 = 43
Count of 2 = 50
Current Entropy is = 0.9959094138937685
Splitting on feature x4 with gain ratio 0.6740904414046132

Level 2
Count of 1 = 42
Count of 2 = 5
Current Entropy is = 0.48890859144051524
Splitting on feature x2 with gain ratio 0.01657050867548726

Level 3
Count of 1 = 15
Count of 2 = 3
Current Entropy is = 0.6500224216483541
Split